## Generic Mapping Types

In [1]:
import re
import sys
import time
import random
import builtins
import collections
from dis import dis
from unicodedata import name
from types import MappingProxyType
from collections.abc import Mapping

In [2]:
my_dict = {}
isinstance(my_dict, Mapping)

True

### What Is Hashable?
An object is hashable if it has a hash value which never changes during its lifetime (it needs a `__hash__()` method), and can be compared to other objects (it needs an `__eq__()` method). Hashable objects which compare equal must have the same hash
value. […]

The atomic immutable types (str, bytes, numeric types) are all hashable. A frozen set is always hashable, because its elements must be hashable by definition. ***A tuple is hashable only if all its items are hashable***.

User-defined types are hashable by default because their hash value is their `id()` and they all compare not equal. If an object implements a custom `__eq__` that takes into account its internal state, it may be hashable only if all its attributes are immutable.

In [3]:
tt = (1, 2, (30, 40))
hash(tt)

-3907003130834322577

In [4]:
tl = (1, 2, [30, 40])
try:
    hash(tl)
except:
    print("TypeError: unhashable type: 'list'")

TypeError: unhashable type: 'list'


In [5]:
tf = (1, 2, frozenset([30, 40]))
hash(tf)

5149391500123939311

In [6]:
a = dict(one=1, two=2, three=3)
b = {'one': 1, 'two': 2, 'three': 3}
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('two', 2), ('one', 1), ('three', 3)])
e = dict({'three': 3, 'one': 1, 'two': 2})
a == b == c == d == e

True

## Handling Missing Keys with setdefault

### Example uses dict.get to fetch and update a list of word occurrences from the index

In [7]:
"""Build an index mapping word -> list of occurrences"""
WORD_RE = re.compile('\w+')
index = {}
with open('zen.txt', encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            # this is ugly; coded like this to make a point
            occurrences = index.get(word, [])
            occurrences.append(location)
            index[word] = occurrences

In [8]:
for word in sorted(index, key=str.upper):
    print(word, index[word])

a [(19, 48), (20, 53)]
Although [(11, 1), (16, 1), (18, 1)]
ambiguity [(14, 16)]
and [(15, 23)]
are [(21, 12)]
aren [(10, 15)]
at [(16, 38)]
bad [(19, 50)]
be [(15, 14), (16, 27), (20, 50)]
beats [(11, 23)]
Beautiful [(3, 1)]
better [(3, 14), (4, 13), (5, 11), (6, 12), (7, 9), (8, 11), (17, 8), (18, 25)]
break [(10, 40)]
by [(1, 20)]
cases [(10, 9)]
complex [(5, 23)]
Complex [(6, 1)]
complicated [(6, 24)]
counts [(9, 13)]
dense [(8, 23)]
do [(15, 64), (21, 48)]
Dutch [(16, 61)]
easy [(20, 26)]
enough [(10, 30)]
Errors [(12, 1)]
explain [(19, 34), (20, 34)]
Explicit [(4, 1)]
explicitly [(13, 8)]
face [(14, 8)]
first [(16, 41)]
Flat [(7, 1)]
good [(20, 55)]
great [(21, 28)]
guess [(14, 52)]
hard [(19, 26)]
honking [(21, 20)]
idea [(19, 54), (20, 60), (21, 34)]
If [(19, 1), (20, 1)]
implementation [(19, 8), (20, 8)]
implicit [(4, 25)]
In [(14, 1)]
is [(3, 11), (4, 10), (5, 8), (6, 9), (7, 6), (8, 8), (17, 5), (18, 16), (19, 23), (20, 23)]
it [(15, 67), (19, 43), (20, 43)]
let [(21, 42)]
m

*The three lines dealing with occurrences in above Example can be replaced by a single
line using `dict.setdefault`.*

In [9]:
index = {}
with open('zen.txt', encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            index.setdefault(word, []).append(location)

for word in sorted(index, key=str.upper):
    print(word, index[word])

a [(19, 48), (20, 53)]
Although [(11, 1), (16, 1), (18, 1)]
ambiguity [(14, 16)]
and [(15, 23)]
are [(21, 12)]
aren [(10, 15)]
at [(16, 38)]
bad [(19, 50)]
be [(15, 14), (16, 27), (20, 50)]
beats [(11, 23)]
Beautiful [(3, 1)]
better [(3, 14), (4, 13), (5, 11), (6, 12), (7, 9), (8, 11), (17, 8), (18, 25)]
break [(10, 40)]
by [(1, 20)]
cases [(10, 9)]
complex [(5, 23)]
Complex [(6, 1)]
complicated [(6, 24)]
counts [(9, 13)]
dense [(8, 23)]
do [(15, 64), (21, 48)]
Dutch [(16, 61)]
easy [(20, 26)]
enough [(10, 30)]
Errors [(12, 1)]
explain [(19, 34), (20, 34)]
Explicit [(4, 1)]
explicitly [(13, 8)]
face [(14, 8)]
first [(16, 41)]
Flat [(7, 1)]
good [(20, 55)]
great [(21, 28)]
guess [(14, 52)]
hard [(19, 26)]
honking [(21, 20)]
idea [(19, 54), (20, 60), (21, 34)]
If [(19, 1), (20, 1)]
implementation [(19, 8), (20, 8)]
implicit [(4, 25)]
In [(14, 1)]
is [(3, 11), (4, 10), (5, 8), (6, 9), (7, 6), (8, 8), (17, 5), (18, 16), (19, 23), (20, 23)]
it [(15, 67), (19, 43), (20, 43)]
let [(21, 42)]
m

In other words, the end result of this line…

```
my_dict.setdefault(key, []).append(new_value)
```

…is the same as running…

```
if key not in my_dict:
    my_dict[key] = []
    my_dict[key].append(new_value)
```

## defaultdict: Another Take on Missing Keys

In [10]:
index = collections.defaultdict(list)
with open('zen.txt', encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            index[word].append(location)

# print in alphabetical order
for word in sorted(index, key=str.upper):
    print(word, index[word])

a [(19, 48), (20, 53)]
Although [(11, 1), (16, 1), (18, 1)]
ambiguity [(14, 16)]
and [(15, 23)]
are [(21, 12)]
aren [(10, 15)]
at [(16, 38)]
bad [(19, 50)]
be [(15, 14), (16, 27), (20, 50)]
beats [(11, 23)]
Beautiful [(3, 1)]
better [(3, 14), (4, 13), (5, 11), (6, 12), (7, 9), (8, 11), (17, 8), (18, 25)]
break [(10, 40)]
by [(1, 20)]
cases [(10, 9)]
complex [(5, 23)]
Complex [(6, 1)]
complicated [(6, 24)]
counts [(9, 13)]
dense [(8, 23)]
do [(15, 64), (21, 48)]
Dutch [(16, 61)]
easy [(20, 26)]
enough [(10, 30)]
Errors [(12, 1)]
explain [(19, 34), (20, 34)]
Explicit [(4, 1)]
explicitly [(13, 8)]
face [(14, 8)]
first [(16, 41)]
Flat [(7, 1)]
good [(20, 55)]
great [(21, 28)]
guess [(14, 52)]
hard [(19, 26)]
honking [(21, 20)]
idea [(19, 54), (20, 60), (21, 34)]
If [(19, 1), (20, 1)]
implementation [(19, 8), (20, 8)]
implicit [(4, 25)]
In [(14, 1)]
is [(3, 11), (4, 10), (5, 8), (6, 9), (7, 6), (8, 8), (17, 5), (18, 16), (19, 23), (20, 23)]
it [(15, 67), (19, 43), (20, 43)]
let [(21, 42)]
m

***The default_factory of a defaultdict is only invoked to provide
default values for `__getitem__` calls, and not for the other
methods. For example, if dd is a defaultdict, and k is a missing
key, `dd[k]` will call the default_factory to create a default value,
but `dd.get(k)` still returns `None`.***

## The __missing__ Method

Underlying the way mappings deal with missing keys is the aptly named \_\_missing\_\_
method. This method is not defined in the base `dict` class, but dict is aware of it: if you
subclass `dict` and provide a `__missing__` method, the standard `dict.__getitem__` will
call it whenever a key is not found, instead of raising `KeyError`.

### StrKeyDict0 converts nonstring keys to str on lookup

In [11]:
class StrKeyDict0(dict):
    
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
    
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()

In [12]:
d = StrKeyDict0([('2', 'two'), ('4', 'four')])

In [13]:
d['2']

'two'

In [14]:
d[4]

'four'

In [15]:
try:
    d[1]
except KeyError:
    print("KeyError: '1'")

KeyError: '1'


In [16]:
d.get(2)

'two'

In [17]:
d.get('4')

'four'

In [18]:
d.get(1, 'N/A')

'N/A'

In [19]:
2 in d

True

In [20]:
1 in d

False

## Variations of dict

In [21]:
pylookup = collections.ChainMap(locals(), globals(), vars(builtins))

In [22]:
ct = collections.Counter('abracadabra')
ct

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

In [23]:
ct.update('aaaaazzz')
ct

Counter({'a': 10, 'b': 2, 'r': 2, 'c': 1, 'd': 1, 'z': 3})

In [24]:
ct.most_common(2)

[('a', 10), ('z', 3)]

## Subclassing UserDict

In [25]:
class StrKeyDict(collections.UserDict):
    """
    A better way to create a user-defined mapping type is to subclass
    collections.UserDict instead of dict
    """
    
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]

    def __contains__(self, key):
        return str(key) in self.data
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item

## Immutable Mappings

In [26]:
d = {1: 'A'}
d_proxy = MappingProxyType(d)
d_proxy

mappingproxy({1: 'A'})

In [27]:
d_proxy[1]

'A'

In [28]:
try:
    d_proxy[2] = 'x'
except TypeError:
    print("TypeError: 'mappingproxy' object does not support item assignment")

TypeError: 'mappingproxy' object does not support item assignment


In [29]:
d[2] = 'B'
d_proxy

mappingproxy({1: 'A', 2: 'B'})

In [30]:
d_proxy[2]

'B'

## Set Theory
A `set` is a collection of unique objects. A basic use case is removing duplication.
***Set elements must be hashable***. The set type is not `hashable`, but `frozenset` is, so you can have frozenset elements inside a set.
In addition to guaranteeing uniqueness, the set types implement the essential set operations as infix operators, so, given two sets a and b, `a | b` returns their union, `a & b` computes the intersection, `a - b` the difference and `a ^ b` the symmetric difference.

### Count occurrences of needles in a haystack, both of type set

In [31]:
needles = [i for i in range(10000) if i % 5 == 0]
haystack = [i for i in range(10000) if i % 3 == 0]
found = len(set(needles) & set(haystack))
found

667

## set Literals

In [32]:
s = {1}
type(s)

set

In [33]:
s.pop()

1

In [34]:
s

set()

In [35]:
dis('{1}')

  1           0 LOAD_CONST               0 (1)
              2 BUILD_SET                1
              4 RETURN_VALUE


In [36]:
dis('set([1])')

  1           0 LOAD_NAME                0 (set)
              2 LOAD_CONST               0 (1)
              4 BUILD_LIST               1
              6 CALL_FUNCTION            1
              8 RETURN_VALUE


In [37]:
frozenset(range(10))

frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

## Set comprehension

In [38]:
{chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i),'')}

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}

## dict and set Under the Hood

In [39]:
s = {*range(1000000)}
l = [*range(1000000)]

In [40]:
%%timeit
563 in s

91.4 ns ± 4.1 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [41]:
%%timeit
563 in l

10.7 µs ± 244 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [42]:
haystack = [i for i in range(1000)]
haystack_s = {i for i in range(1000)}
haystack_d = {i: str(i) for i in range(1000)}

In [43]:
needle = random.randint(0, 1000)
needle

235

In [44]:
%timeit 552 in haystack 
%timeit 552 in haystack_s
%timeit 552 in haystack_d

KeyboardInterrupt: 

In [ ]:
for size in [1_000, 10_000, 100_000, 1_000_000, 10_000_000]:
    needle = random.randint(0, size)
    haystack = [i for i in range(size)]
    haystack_s = {i for i in range(size)}
    haystack_d = {i: str(i) for i in range(size)}
    ts = time.perf_counter()
    is_present = needle in haystack
    print(f"Search time in {size} elements: List -> {time.perf_counter() - ts}")
    ts = time.perf_counter()
    is_present = needle in haystack_s
    print(f"Search time in {size} elements: Set -> {time.perf_counter() - ts}")
    ts = time.perf_counter()
    is_present = needle in haystack_d
    print(f"Search time in {size} elements: Dict -> {time.perf_counter() - ts}")
    print()

## Key ordering depends on insertion order

In [ ]:
DIAL_CODES = [
    (86, 'China'),
    (91, 'India'),
    (1, 'United States'),
    (62, 'Indonesia'),
    (55, 'Brazil'),
    (92, 'Pakistan'),
    (880, 'Bangladesh'),
    (234, 'Nigeria'),
    (7, 'Russia'),
    (81, 'Japan'),
]
d1 = dict(DIAL_CODES)
print('d1:', d1.keys())
d2 = dict(sorted(DIAL_CODES))
print('d2:', d2.keys())
d3 = dict(sorted(DIAL_CODES, key=lambda x:x[1]))
print('d3:', d3.keys())
assert d1 == d2 and d2 == d3

## Adding items to a dict may change the order of existing keys

***This is why modifying the contents of a dict while iterating through it is a bad idea***.